# Advanced data processing with Pandas

In this week, we will continue developing our skills using Pandas to analyze climate data. The aim of this lesson
is to learn different functions to manipulate with the data and do simple analyses. In the end, our goal is
to detect weather anomalies (stormy winds) in Helsinki, during August 2017.

## Reading the data

Notice that this time, we will read the **actual** data obtained from NOAA
without any modifications to the actual data by us. The data is separated with varying amount of spaces (fixed width). The first lines and columns of the data looks like following:

```
      USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP  ...
    029740 99999 201708040000 114   6 *** *** BKN * * * 25.0 03 ** ** ** ** ** ** ** 2   58   56 1005.6  ...
```

Because the data is separated with varying amount of spaces, we need to tell Pandas how to read it. We can control the delimiter with ``sep`` parameter following the documentation of the function `read_csv()`:

![](../img/read-csv-varying-spaces.PNG)

Hence, we can separate the columns by varying number spaces of spaces with ``sep='\s+'`` -parameter.
Our data also included No Data values with varying number of ``*`` -characters. Hence, we need to take also those
into account when reading the data. We can tell Pandas to consider those characters as NaNs by specifying ``na_values=['*', '**', '***', '****', '*****', '******']``.

- Let's start by reading the data with pandas:

## Exploring data and renaming columns

- Let's see how the data looks by printing the first five rows with ``head()`` function

Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterix (\*) characters into `NaN` -values. 

- Let's continue and check what columns do we have.

As we can see, there are quite many columns, however, we are not interested to use all of them.

- Let's select only columns that might be used to detect unexceptional weather conditions, i.e. YR--MODAHRMN, DIR, SPD, GUS, TEMP, MAX, and MIN.


- Let's see what our data looks like now by printing **last** 5 rows and the datatypes.

The column names that we have are somewhat ackward. Let's change them into more intuitive ones. 
This can be done easily with ``rename()`` -function.

We can define the new column names by using a specific data type in Python called [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we determine "`key: value`" -pairs, in which the original column name (the one which will be replaced) is the key, and the new column name is the value.

- Let's change:
   
   - ``YR--MODAHRMN`` column into ``TIME``, 
   - ``SPD`` into ``SPEED``, and
   - ``GUS`` into ``GUST``


From above we can see that we have successfully created a dictionary that is of type `dict`. 

- Now we can change the column names by passing that dictionary into parameter ``columns`` in ``rename()`` -function.

Perfect, now our column names are more easy to understand and use. 

- Let's check some basic statistics to understand our data better.

Okey so from here we can see that there are varying number of observations per column (see the `count` -information). 

For example **`SPD`** and **`TEMP`** column has 72 observations whereas **`GUS`** has only 20 observations and **`MAX`** and **`MIN`** has only 2 observations. From here we can already guess that `MAX` and `MIN` attributes are most probably not going to be useful for us.
However, `GUS` might be.

- Let's explore further our data by checking the first 30 rows of it.



Okey, so from here we can actually see that the **`GUST`** column contains information only on an hourly level. That might be useful! Let's keep this in mind.

**TAKE HOME MESSAGE**: Whenever starting a data analysis with new dataset, it is highly useful to explore the data by calculating basic statistics from the data (+ visualizing the data, which we will learn later). 

## Iterating rows and using self-made functions in Pandas

Let's do the "SAME THING" as so many times before and convert our Fahrenheit temperatures into Celsius (sorry if we seem to lack imagination =) ).

In this time, however, we will use our self-made function to do the conversion.

- Let's first define the function:

In [ ]:
def fahrToCelsius(temp_fahrenheit):
    """
    Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius and return it
    converted_temp = (temp_fahrenheit - 32) / 1.8
    return converted_temp

Let's do the conversion by iterating our data line by line and updating a column called **`CELSIUS`** that we will create.

We can iterate over the rows of Pandas DataFrame by using **`iterrows()`** -function.
When iterating over the rows in our `DataFrame`, it is noteworthy to understand that the Pandas actually keeps track on the **`index`** value as well. Hence, the contents of a single row actually contains not only the values, but also the `index` of that row.

- Let's see how it works. Here, we will use a specific Python command called [**`break`**](https://www.tutorialspoint.com/python/python_break_statement.htm) can be used to stop the iteration right after the first loop. This can be quite useful as we don't want to fill our console by printing all the values and indices in our DataFrame, but to just see if the function works as we want:


Okey, so here we can see that the **`idx`** variable indeed contains the index value at position 0 (the first row) and the **`row`** variable contains all the data from that given row stored as **`pd.Series`**.

- Let's now create an empty column for the Celsius temperatures and update the values into that column by using our function. Here is the whole procedure:


- Let's see what we have now.

Great! Now we have converted our temperatures into Celsius by using the function that we created ourselves.
Knowing how to use your own function in Pandas can be really useful when doing your own analyses. There is also another more powerful way of using functions in Pandas by taking advantage of [**`apply()`**](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) -function, but we will learn that later.

- Finally, let's convert the wind speeds into meters per second values (m/s) as they are more familiar to us in Finland. This can be done with a formula **`m/s = mph x 0.44704`**:


## String manipulation in Pandas

When looking the data more carefully, we can see something interesting:
 - **`GUST`** seems to be measured only once an hour, whereas **`SPD`** (wind speed), and our temperatures seem to be measured approximately every 20 minutes (at minutes XX:00, XX:20 and XX:50).

That might be a problem as we might not be able to compare e.g. the average wind speeds and the speeds during the gust together as they are measured with different intervals. This kind of mismatch between sampling rates of measurements is actually quite typical when working with real data.

How we can solve this problem, is to aggregate the wind speeds into hourly level data so that the attributes become comparable.
First we need to be able to group the values by hour. This can be done e.g. by slicing the date+hour time from the **`TIME`** -column (i.e. removing the minutes from the end of the value).

Doing this requires two steps:
  1. Convert the `TIME` column from `int` into `str` datatype.
  2. Include only numbers up to hourly accuracy (exclude minutes) by slicing texts

**Note:** There are also more advanced functions in Pandas to do time series manipulations by utilizing [**`datetime`**](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) datatype and [**`resample()`**](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#resampling>) -function, but we won't cover those here. You can read the Pandas docs if you are interested.

- Let's convert the time into string. And check that the data type changes:

Okey it seems that now we indeed have the `TIME` as `str` datatype as well.

- Now we can slice them into hourly level by including only 10 first characters from the text (i.e. excluding the minute-level information).


Nice! Now we have information about time on an hourly basis including the date as well.

**Note:** All the typical ``str`` functionalities can be applied to Series of text data with syntax `data['mySeries'].str.<functionToUse>()`.

- Let's also slice only the hour of the day (excluding information about the date) and convert it back to integer (we will be using this information later):

**Wunderbar**, now we have also a separate column for only the hour of the day!

## Aggregating data in Pandas by grouping

Next we want to calculate the average temperatures, wind speeds, etc. on an hourly basis to enable us
to compare all of them to each other.

This can be done by aggregating the data, i.e.:

  1. **grouping the data** based on hourly values
  2. Iterating over those groups and calculating the average values of our attributes
  3. Inserting those values into a new DataFrame where we store the aggregated data

- Let's first create a new **empty** DataFrame where we will store our aggregated data


- Let's then **group** our data based on `TIME_h` attribute that contains the information about the date + hour.

- Let's see what we have now.

Okey, interesting. Now we have a new object with type **`DataFrameGroupBy`**. And it seems that we have 24 individual groups in our data, i.e. **one group for each hour of the day**.

As you might have noticed earlier, the first hour in hour data is `2017080400` (midnight at 4th of August in 2017).

- Let's now see what we have on hour `grouped` variable e.g. on the first hour `2017080400`.
We can get the values of that hour from `DataFrameGroupBy` -object with **`get_group()`** -function.


In [ ]:
# Specify the time of the first hour (as text)
time1 = '2017080400'

Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific hour.
This is really useful, because now we can calculate e.g. the average values for all weather measurements (+ hour) that we have (you can use any of the statistical functions that we have seen already, e.g. mean, std, min, max, median, etc.).

We can do that by using the **`mean()`** -function that we already used during the Lesson 5. 

- Let's calculate the mean for following attributes (let's see how to do them all at once!): 
   - ``DIR``, 
   - ``SPEED``, 
   - ``GUST``, 
   - ``TEMP``, 
   - ``Celsius``.

Nice, now we have averaged our data and e.g. the mean Celsius temperature seems to be about right when comparing to the original values above. As you saw from this example, it is possible to do calculations for multiple columns at the same time efficiently with Pandas.

Notice that we still have information about the hour but not about the date which is at the moment stored in **`time1`** -variable.
We can insert that datetime-information into our **`mean_values`** Series so that we have the date information also associated with our data.


Perfect! Now we have also time information there. 

The last thing to do is to add these mean values into our DataFrame that we created.
That can be done with **`append()`** -function in a quite similar manner as with Python lists. In Pandas the data insertion is not done **inplace** (as when appending to Python lists) so we need to specify that we are updating the aggr_data (using the **`=`** sign). We also need to specify that we ignore the index values of our original DataFrame (i.e. the indices of `mean_values`).


As we can see, now we have a single row in our new DataFrame where we have aggregated the data based on hourly mean values.
Next we could continue doing and insert the average values from other hours in a similar manner but, of course, that is not
something that we want to do manually (would require repeating these same steps too many times).
Luckily, we can actually iterate over all the groups that we have in our data and do these steps using a **`for`** -loop.

When iterating over the groups in our **`DataFrameGroupBy`** -object
it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the **`key`** that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using **`break`** to only see what is happening).


Okey so from here we can see that the **`key`** contains the value **`2017080400`** that is the same
as the values in **`TIME_dh`** column. Meaning that we, indeed, grouped the values based on that column.

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeate slightly the earlier steps so that you can see and better understand what is happening.


- Let's see what we have now:

Awesome! Now we have aggregated our data based on daily averages and we have a new DataFrame called **`aggr_data`** where all those aggregated values are stored.

## Finding outliers from the data

Finally, we are ready to do some real data analytics and check whether we are able to find out if there are any outliers in our data suggesting to have a storm (meaning strong winds in this case).

Here, we define an outlier if the **wind speed is 2 times the standard deviation higher than the average wind speed** (column `SPEED`).

- Let's first find out what is the standard deviation and the mean of the Wind speed.


Okey, so the variance in the windspeed tend to be approximately 1.6 meters per second, and the wind speed is approximately 5.2 m/s. 

- Hence, the threshold for a wind speed to be an outlier with our criteria is:


- Let's finally create a column called **`Outlier`** which we update with **`True`** value, if the windspeed is an outlier, and **`False`**, if it is not. We do this again by iterating over the rows.


Okey now we have at least many False values in our **`Outlier`** -column but there seems to be also one True!.

- Let's select the rows with potential storm:


Okey, so indeed, there was one outlier in our data but the wind during that time wasn't that strong as the average speed was only approximately 9 m/s. This is not too strange as we were only looking at data from a single day.

## Repeating the data analysis with larger dataset

Let's continue by executing the steps that we have written this far and use it to explore outlier winds based on whole month of August 2017.

For this purpose, we change the input file to be **`6591337447542dat_August.txt`** that [looks like this](data/6591337447542dat_August.txt).

- Here we will repeat all the steps that we did earlier in one code block so that you can see the full picture:

In [ ]:
# Filepath
fp = "data/6591337447542dat_August.txt"

After running the analysis with our new dataset, let's explore and see we have.

- Let's start by checking if the average and standard deviation of the windspeed differ from the previous ones

Okey so they are indeed different now! With larger dataset the average wind speed is 4.2 m/s (compared to 5.2 m/s previously). 

- Let's see what we have now in our **`storm`** -variable.

Okey, interesting! Now we can see the the days and hours when it has been stormy in August 2017.
It seems that the storms have usually been during the day time. Let's check if this is the case.

We can easily count how many stormy observations for different hour of the day there has been by
using a [**`value_counts()`**](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) -function that calculates how many observations per certain value there are in a certain column (works best for categorigal data).

- Let's see the counts for different hours of the day


Okey, this is interesting. It seems that most often it has been stormy at 14:00 GMT (i.e. 16:00 at Finnish time).
Notice, that there haven't been any strong winds during the night, which is also interesting. However, as the **The weather guys** explains us, [it is not that surprising actually](http://wxguys.ssec.wisc.edu/2013/11/18/why-does-the-wind-diminish-after-sunset) =). 

The average wind speed may not be the perfect measure to find extreme weather conditions. Gust might usually be a better measure for that purpose.

- Let's see what were the strongest gust winds in our dataset by sorting the values using **`sort_values()`** -function.


Interesting! There was one hour with quite extraordinary gust wind in our data happening at 12th of August in 2017.
Indeed, that was a big storm in Helsinki called [**Kiira**](https://yle.fi/uutiset/osasto/news/saturday_night_storm_downs_trees_cuts_electricity_in_the_south/9773250) that caused major damage in different parts of the city. (*Source: [YLE](https://yle.fi/uutiset/osasto/news/saturday_night_storm_downs_trees_cuts_electricity_in_the_south/9773250); 
Photo: Markku Sipi*)


![](../img/Kiira-storm.PNG)


**TAKE HOME MESSAGE**: As we have seen here, we can already conduct fairly interesting data analysis with Pandas that provides various useful functionalities that are fairly straightforward and easy to use. Similar approaches can be used for many different kind of datasets. 